In [ ]:
# setup
import hockey_scraper
import requests
import pandas as pd
import uuid
from py2neo import Graph, Node, Relationship, NodeMatcher


In [ ]:
# setup neo4j
driver = Graph(user="neo4j", password="password")
driver.schema.create_uniqueness_constraint("Game", "pk")
driver.schema.create_uniqueness_constraint("Venue", "id")
driver.schema.create_uniqueness_constraint("Play", "id")
driver.schema.create_uniqueness_constraint("Player", "id")

matcher = NodeMatcher(driver)

In [ ]:
# seutp mongo
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client['nhl']
collection = db['pbp']

In [ ]:
# get the 2018/19 gameids
URL = "http://www.nicetimeonice.com/api/seasons/20182019/games"
resp = requests.get(URL)
data = resp.json()

# list of games
games = pd.DataFrame(data)
gids = list(games.gameID)

In [ ]:
# get the json document and throw it into mongodb
for gid in gids:
    GURL = "http://statsapi.web.nhl.com/api/v1/game/{}/feed/live".format(gid)
    resp = requests.get(GURL)
    game = resp.json()
    collection.insert_one(game)
    print("finished game id {}".format(str(gid)))
    


## Parse the game the and the plays

In [35]:
# for each game (document) parse into neo4j
i = 1

for game in collection.find(no_cursor_timeout=True):
    
    try:
        #### game, team, and players for the game
        #### https://py2neo.org/v4/database.html?highlight=merge#py2neo.database.Graph.merge
        #### ^^ primary label and key for merge
        gdata = game['gameData']
        gid = gdata['game']['pk']
        print("starting {}".format(str(gid)))

        ## create the game node
        _game = gdata['game']
        _game.update(gdata['datetime'])
        g = Node.cast(_game)
        g.add_label('Game')
        driver.create(g)

        ## parse the venue
        venue = gdata['venue']
        if 'id' not in venue.keys():
            venue['id'] = str(uuid.uuid1())
        v = Node.cast(venue)
        v.add_label('Venue')
        driver.merge(v, "Venue", "id")
        rgv = Relationship(g, "PLAYED_AT", v)
        driver.create(rgv)

        ## parse the teams
        #teams = gdata['teams']
        #_away = teams['away']
        #away = Node.cast(_away)
        #away

        #### live data (plays and players in the play)

        ## get the plays
        plays = game['liveData']['plays']['allPlays']

        # build the plays
        for index, play_data in enumerate(plays):
            play = play_data['result']
            play.update(play_data['about'])
            if 'goals' in play.keys():
              del play['goals']    
            if 'strength' in play.keys():
              del play['strength']    
            play.update(play_data['about']['goals'])
            play.update(play_data['coordinates'])
            play['id'] = "{}_{}".format(gid, index)
            play['gid'] = gid
            play['pid'] = index
            p = Node.cast(play)  ## cast from a dict
            p.add_label("Play") ## add the label I want
            driver.create(p) ## create the node conditionally with trail
            if index == 0:
              r = Relationship(g, "HAD_PLAY", p)
              driver.create(r)
            if index >= 1:
              p0 = matcher.match("Play", pid = index - 1, gid =  gid).first()
              r = Relationship(p0, "NEXT_PLAY", p)
              driver.create(r)
            # parse the players
            if "players" in play_data.keys():
                players = play_data['players']
                for player in players:
                    pnode = player['player']
                    del player['player']
                    player.update(pnode)
                    del pnode
                    # build the node and the relationship
                    player_node = Node.cast(player)
                    player_node.add_label("Player")
                    driver.merge(player_node, "Player", "id")
                    r_play = Relationship(p, player['playerType'], player_node)
                    driver.create(r_play)


        # status
        print("finished {}".format(str(gid)))
    #     i = i + 1
    #     if i == 5:
    #         break
    except:
        print("skipping a game")
        pass


starting 2018030417
skipping a game
starting 2018030416
skipping a game
starting 2018030415
skipping a game
starting 2018030414
skipping a game
starting 2018030413
skipping a game
starting 2018030412
skipping a game
starting 2018030411
skipping a game
starting 2018030326
skipping a game
starting 2018030325
skipping a game
starting 2018030324
skipping a game
starting 2018030314
skipping a game
starting 2018030323
skipping a game
starting 2018030313
skipping a game
starting 2018030322
skipping a game
starting 2018030312
skipping a game
starting 2018030321
skipping a game
starting 2018030311
skipping a game
starting 2018030247
skipping a game
starting 2018030237
skipping a game
starting 2018030246
skipping a game
starting 2018030216
skipping a game
starting 2018030236
skipping a game
starting 2018030245
skipping a game
starting 2018030215
skipping a game
starting 2018030235
skipping a game
starting 2018030224
skipping a game
starting 2018030244
skipping a game
starting 2018030214
skipping

CursorNotFound: cursor id 18748399537 not found

> didnt run the code with try/except, it takes a awhile.

---

In [ ]:
venue = gdata['venue']

In [ ]:
venue

In [ ]:
venue.keys()

In [ ]:
if 'id' not in venue.keys():
    venue['id'] = str(uuid.uuid1())

In [ ]:
venue

---

In [ ]:
game = collection.find_one()

In [ ]:
game.keys()

In [ ]:
game['metaData']

In [ ]:
game['gameData'].keys()

In [ ]:
gdata = game['gameData']

In [ ]:
gdata['game']

In [ ]:
game['gamePk']

In [ ]:
gdata['datetime']

In [ ]:
gdata['venue']

In [ ]:
gdata['teams']['away']

In [ ]:
gdata = game['gameData']
teams = gdata['teams']

In [ ]:
away_team = teams['away']

In [ ]:
away_team

In [ ]:
_away = teams['away']

In [ ]:
away = Node.cast(_away)

In [ ]:
away

In [ ]:
gdata = game['gameData']

In [ ]:
gdata['game']['pk']

In [ ]:
bad = '2018030322'

In [ ]:
gid

In [ ]:
gdata['venue']
